In [1]:
#!pip install celery

In [2]:
#!sudo rabbitmqctl add_user myguest myguestpwd
#!sudo rabbitmqctl set_permissions -p / myguest "." "." ".*"

In [3]:
!sudo rabbitmqctl list_users

Listing users ...
guest	[administrator]
myguest	[]


This code will **run on a server machine**. It will ask worker machines to complete some tasks and send the results back to the server.

In the same folder with this code there is a test.py file 
You will need to run that code on a **worker** machines e.g. with <br>

**"celery -A test worker --loglevel=info --concurrency=3" on worker machine 1**
**"celery -A test worker --loglevel=info --concurrency=2" on worker machine 2**

Then those machines will become workers, and will be able to run the app task, i.e. in this case the echo function, whenever the broker requests it.

The echo function will be run by these commands on the remote (=worker) machine:

from test import echo
res = echo.delay('Python rocks!'); print(type(res)); print(res)
res.result

In [4]:
from test import echo

Before you run the next cell, you might need to <br>
"pip install celery" on the worker machine and then run <br>
"celery -A test worker --loglevel=info" on the worker machine.

Otherwise the result of 'res1.ready()' in the cell below will be False forever

**Check the available worker machines** <br>
In the "!celery -A test status" command we use "test" because "test.py" runs on the workers

In [14]:
!celery -A test status

celery@ip-172-31-3-168: OK
celery@ip-172-31-3-202: OK

2 nodes online.


In [5]:
# We will run the echo function on the worker machine.
# The fun.delay(arg) function sends the arg message to the worker's fun function and run it on the worker machine.
# For more info, see https://docs.celeryproject.org/en/latest/userguide/calling.html
    
# This code will send 'Python rocks 1 :) !' to the worker machine, and put its response into res1
res1 = echo.delay('Python rocks 1 :) !'); print(type(res1)); print(res1)

<class 'celery.result.AsyncResult'>
ad3f33f5-1da9-4b94-9a4c-fd493253f33e


In [6]:
res1.ready()

False

In [7]:
res1.result

**Now let us run a series of jobs asynchronously on the workers. Observe the round-robin job allocation**

In [8]:
res =[]
for iter in range(1,30):
    res.append(echo.delay('hello ' + str(iter)))

**Let us wait for the results with ".get()", and print them out**

In [9]:
for iter in range(len(res)):
    print(res[iter].get()) #let us wait for the results and print them out

Response from worker: 172.31.3.168 process_name: ForkPoolWorker-2 process_index: 1 os_pid: 24674 message: hello 1 **
Response from worker: 172.31.3.202 process_name: ForkPoolWorker-2 process_index: 1 os_pid: 1556 message: hello 2 **
Response from worker: 172.31.3.168 process_name: ForkPoolWorker-3 process_index: 2 os_pid: 24675 message: hello 3 **
Response from worker: 172.31.3.202 process_name: ForkPoolWorker-1 process_index: 0 os_pid: 1555 message: hello 4 **
Response from worker: 172.31.3.168 process_name: ForkPoolWorker-1 process_index: 0 os_pid: 24671 message: hello 5 **
Response from worker: 172.31.3.202 process_name: ForkPoolWorker-1 process_index: 0 os_pid: 1555 message: hello 6 **
Response from worker: 172.31.3.168 process_name: ForkPoolWorker-2 process_index: 1 os_pid: 24674 message: hello 7 **
Response from worker: 172.31.3.202 process_name: ForkPoolWorker-1 process_index: 0 os_pid: 1555 message: hello 8 **
Response from worker: 172.31.3.168 process_name: ForkPoolWorker-1 pr

**Let us run the echo function on the server instead of the workers:**

In [10]:
res=echo('running on the server')

172.31.3.168 process_name: MainProcess process_index: _none_ os_pid: 25009 message: running on the server **
